In [1]:
imgsize = 32

In [2]:
from __future__ import print_function
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "5"
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from tqdm.notebook import tqdm
from six.moves import urllib

torch.manual_seed(0)
plt.ion()  
import tensorflow
import keras
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Model, Sequential
from keras.layers import Dense, Conv2D, Dropout, BatchNormalization, Input, Reshape, Flatten
# from keras.optimizers import Adam
from tensorflow.keras.optimizers import Adam
import os
from sklearn.svm import SVC
import scipy.linalg as la
from numpy.random import seed
import gurobipy as gp
from gurobipy import GRB
seed(1)
tensorflow.random.set_seed(2)


2022-10-09 23:35:46.171560: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-09 23:35:46.378318: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-09 23:35:46.378350: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-09 23:35:46.408922: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-09 23:35:47.280132: W tensorflow/stream_executor/pla

In [3]:
opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]
urllib.request.install_opener(opener)

device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")

full_dataset = torchvision.datasets.ImageFolder(root = '/data/siddhant/MedicalSTN/DrVineetData/Knee_Full_AP', 
                                     transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Grayscale(num_output_channels=3),
                       transforms.Resize((imgsize,imgsize)),
                   ]))

X, y = torch.vstack([full_dataset[i][0].unsqueeze(0) for i in range(len(full_dataset))]), torch.tensor(full_dataset.targets)
# train_size = int(0.8 * len(full_dataset))
# test_size = len(full_dataset) - train_size
# train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])

# # Training dataset
# train_loader = torch.utils.data.DataLoader(
#     train_dataset, 
#     batch_size=64, shuffle=True, num_workers=2)
# # Test dataset
# test_loader = torch.utils.data.DataLoader(
#     test_dataset, 
#     batch_size=64, shuffle=True, num_workers=2)

# outputlen = len(full_dataset.find_classes('/data/siddhant/MedicalSTN/DrVineetData/Knee_Full_AP')[0])

In [4]:
Xshape = X.shape
Xshape

torch.Size([747, 3, 32, 32])

In [5]:
#optimization problem to get the high dimensional data point from low dimensional data point
#datapoint_low -> low dimensional data point
#G -> projection matrix
#lb -> lower bounds of the features
#ub -> upper bounds of the features
#C,epsilon -> constants in the optimization problem
#N -> high dimensional feature size
#zi_type -> vector of size N representing whether a feature is integer value(1) or real value(0)
def GetHighDimensionDataset(datapoint_low, G, lb, ub, C, epsilon, N, zi_type):
    datapoint_low = datapoint_low.reshape(datapoint_low.shape[0],)
    lb = lb.reshape(lb.shape[0])
    ub = ub.reshape(ub.shape[0])
    K = datapoint_low.shape[0]
    eps_vector = epsilon * np.ones((K,))
    zi_type = zi_type.reshape(-1)

    attr_index = np.arange(zi_type.shape[0])
    zi_int_index = attr_index[zi_type==1]
    zi_real_index = attr_index[zi_type==0]
    zi_int_lb = lb[zi_int_index]
    zi_int_ub = ub[zi_int_index]
    zi_real_lb = lb[zi_real_index]
    zi_real_ub = ub[zi_real_index]
    G_int = G[zi_int_index,:]
    G_real = G[zi_real_index,:]

    #using gurobipy   
    if zi_int_index.shape[0]==0 and zi_real_index.shape[0]!=0: #only real attr
      model = gp.Model("eigenbag1")
      model.Params.IterationLimit = iter_lim #GRB.INFINITY
      zi_real = model.addMVar ( zi_real_index.shape[0], vtype=GRB.CONTINUOUS, name="zi_real" )
      qip = model.addMVar ( K, lb=0.0, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="qip" )
      qim = model.addMVar ( K, lb=0.0, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="qim" )
      model.addConstr( (G_real.T)@zi_real - qip <= datapoint_low + eps_vector, name="c1")
      model.addConstr( -(G_real.T)@zi_real - qim <= -datapoint_low + eps_vector, name="c2")
      model.addConstr( zi_real <= zi_real_ub , name="c3")
      model.addConstr( zi_real >= zi_real_lb , name="c4")

      model.setObjective(( C*qip.sum() + C*qim.sum() ), sense=GRB.MINIMIZE)
      # Verify model formulation
      model.write('eigenbag1.lp')
      # Run optimization engine
      model.optimize()
      try:
        zi_real = (zi_real.x)
        zi_int = np.array([])
      except:
        return np.random.rand(N, 1),1
  
    elif zi_int_index.shape[0]!=0 and zi_real_index.shape[0]==0: #only int attr
      model = gp.Model("eigenbag1")
      model.Params.IterationLimit = iter_lim #GRB.INFINITY
      zi_int = model.addMVar ( zi_int_index.shape[0], vtype=GRB.INTEGER, name="zi_int" )
      qip = model.addMVar ( K, lb=0.0, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="qip" )
      qim = model.addMVar ( K, lb=0.0, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="qim" )
      model.addConstr( (G_int.T)@zi_int - qip <= datapoint_low + eps_vector, name="c1")
      model.addConstr( -(G_int.T)@zi_int - qim <= -datapoint_low + eps_vector, name="c2")
      model.addConstr( zi_int <= zi_int_ub , name="c3")
      model.addConstr( zi_int >= zi_int_lb , name="c4")

      model.setObjective(( C*qip.sum() + C*qim.sum() ), sense=GRB.MINIMIZE)
      # Verify model formulation
      model.write('eigenbag1.lp')
      # Run optimization engine
      model.optimize()
      try:
        zi_int = (zi_int.x)
        zi_real = np.array([])
      except:
        return np.random.rand(N, 1),1

    else: #both real and int attr
      model = gp.Model("eigenbag1")
      model.Params.IterationLimit = iter_lim#1000000.0#GRB.INFINITY
      zi_real = model.addMVar ( zi_real_index.shape[0], vtype=GRB.CONTINUOUS, name="zi_real" )
      zi_int = model.addMVar ( zi_int_index.shape[0], vtype=GRB.INTEGER, name="zi_int" )
      qip = model.addMVar ( K, lb=0.0, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="qip" )
      qim = model.addMVar ( K, lb=0.0, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="qim" )
      model.addConstr( (G_int.T)@zi_int + (G_real.T)@zi_real - qip <= datapoint_low + eps_vector, name="c1")
      model.addConstr( -(G_int.T)@zi_int -(G_real.T)@zi_real - qim <= -datapoint_low + eps_vector, name="c2")
      model.addConstr( zi_int <= zi_int_ub , name="c3")
      model.addConstr( zi_int >= zi_int_lb , name="c4")
      model.addConstr( zi_real <= zi_real_ub , name="c5")
      model.addConstr( zi_real >= zi_real_lb , name="c6")

      model.setObjective(( C*qip.sum() + C*qim.sum() ), sense=GRB.MINIMIZE)
      # Verify model formulation
      model.write('eigenbag1.lp')
      # Run optimization engine
      model.optimize()
      try:
        zi_int = (zi_int.x)
        zi_real = (zi_real.x)
      except:
        return np.random.rand(N, 1),1

    #rounding the values to nearest integers for integer attributes and making sure the projected back attributes lie between the lower and upper bounds
    zi_int_rounded = np.empty(shape=[0])
    zi_real_rounded = np.empty(shape=[0])
    for i in range(zi_int.shape[0]):
      st_list = [st for st in range(int(zi_int_lb[i]),int(zi_int_ub[i]+1),1)]
      dist = [abs(zi_int[i]-st) for st in range(int(zi_int_lb[i]),int(zi_int_ub[i]+1),1)]
      minpos = dist.index(min(dist))
      rz = np.array([st_list[minpos]])
      zi_int_rounded = np.append(zi_int_rounded,rz,axis=0)
    for i in range(zi_real.shape[0]):
      if zi_real[i] >= zi_real_ub[i]:
        rz = zi_real_ub[i]
      elif zi_real[i] <= zi_real_lb[i]:
        rz = zi_real_lb[i]
      else:
        rz = zi_real[i]
      zi_real_rounded = np.append(zi_real_rounded,np.array([rz]),axis=0)

    zi_unsorted = np.hstack((zi_int_rounded,zi_real_rounded))
    zi_index_unsorted = np.hstack((zi_int_index,zi_real_index))
    zi =  np.empty(shape=[0])
    for i in range(zi_unsorted.shape[0]):
      zi = np.append(zi,zi_unsorted[zi_index_unsorted==i],axis=0)

    datapoint_high = zi
    return datapoint_high,0

In [6]:
#V -> eigenvectors
#D -> eigenvalues
def SortEigenvalues(V, D):
    # sort matrix of eigenvalues
    idx = D.argsort()[::-1]
    # obtain indices of sorted eigenvalues
    D_sort = D[idx]
    # arrange the eigenvectors in the order of sorted eigenvalues
    V_sort = V[:,idx]
    return V_sort, D_sort

In [7]:
#input_dataset -> projected dataset belonging to the cluster
#cluster_center -> cluster center
def FindMidPointDistancesFromClusterCenters(input_dataset, cluster_center):
    # Compute size of dataset
    N, D = input_dataset.shape[0],input_dataset.shape[1]
    # Initialize output dataset matrix
    output_dataset = np.empty(shape = (N,D))
    # Iterate over samples
    for i in range(N):
        # compute the sample as the mid point of the line joining the sample with the corresponding cluster center
        output_dataset[i,:] = np.add(input_dataset[i,:],cluster_center)/2
    return output_dataset

In [8]:
#EVs_vec -> eigenvectors
#eigvar -> required minimum variance
def FindBestEVs(EVs_vec,eigvar): 
    # Compute ratio of variance preserved by eigenvalues
    if eigvar==1:
      sel_EVs = len(EVs_vec)
      return sel_EVs
    else:
      for i in range(len(EVs_vec)):
        sum_ratio = sum([x**2 for x in EVs_vec[0:i+1]]) / sum([x**2 for x in EVs_vec])
        # check if variance captured is > eigvar
        if (sum_ratio >= eigvar):
          # select i eigenvectors
          sel_EVs = i+1
          return sel_EVs

In [9]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from numpy import cov
import collections
#features,labels -> X,y
#Cparam,epsilon -> constants for optimization problem
#zi_type -> vector of size N representing whether a feature is integer value(1) or real value(0)
#eigvar -> required minimum variance
def AugmentDataset(features, labels, Cparam, epsilon, zi_type, eigvar):
    # Compute dataset size
    N, dim = features.shape[0],features.shape[1]
    
    zi_lb = np.min(features,axis=0)
    zi_ub = np.max(features,axis=0)

    corelmatrix = np.corrcoef(features.T)
    corelmatrix[np.isnan(corelmatrix)] = 0    # Sanity check!!
    # Compute eigenvalues and eigenvectors of correlation matrix
    D, V = la.eig(corelmatrix)
    # Sort the eigenvectors in order of decreasing eigenvalues
    V1, D1 = SortEigenvalues(V, D)
    # Choose the k largest eigenvalues and their corresponding eigenvectors
    k = FindBestEVs(D1,eigvar)
    V_selected = V1[:, 0:k]
    # Compute projections of the samples onto the k eigendirections
    projected_dataset = features @ V_selected
    projected_dataset = np.real(projected_dataset)
    # Choose number of clusters by rule of thumb
    k_clusters = math.floor(math.sqrt(N / 2))
    # Perform k-means clustering
    kms=KMeans(n_clusters=k_clusters)
    kms.fit(projected_dataset)
    IDX = kms.predict(projected_dataset)
    C =kms.cluster_centers_
    # construct lines joining the cluster centres with each point of the
    # respective cluster, and then compute the mid-points of these lines
    new_dataset = np.array([])
    new_labels = np.array([])
    for i in range(k_clusters):
        new_dataset_for_cluster = FindMidPointDistancesFromClusterCenters(projected_dataset[IDX == i,:], C[i, :])
        if new_dataset.shape[0]==0:
            new_dataset = new_dataset_for_cluster
        else:
            new_dataset = np.vstack((new_dataset, new_dataset_for_cluster ))

    new_dataset_high_dimension = np.empty(shape=(new_dataset.shape[0],dim))
    for i in range(new_dataset.shape[0]):
        newhd,flag = GetHighDimensionDataset(new_dataset[i, :], V_selected, zi_lb, zi_ub, Cparam, epsilon, dim, zi_type)
        if flag==0:
          new_dataset_high_dimension[i, :] = newhd
    
    #finding the class of the generated samples
    if len(np.unique(labels.reshape(-1)))>1:
      sc = StandardScaler().fit(features)
      features_sc = sc.transform(features)
      new_dataset_high_dimension_sc = sc.transform(new_dataset_high_dimension)
      model = RandomForestClassifier(n_estimators=1000, class_weight='balanced')
      model.fit(features_sc, labels)
      new_labels = (model.predict(new_dataset_high_dimension_sc)).reshape(-1,1)
    else:
      print('lab classes:'+str(np.unique(labels.reshape(-1))))
      new_labels = labels[0][0]*np.ones((new_dataset.shape[0],1))
    
    return new_dataset_high_dimension, new_labels

In [10]:
import random
def random_combination(iterable,r,number_of_comb):
    i = 0
    pool = tuple(iterable)
    n = len(pool)
    rng = range(n)
    while i < number_of_comb:
        i += 1
        yield [pool[j] for j in random.sample(rng, r)] 

In [11]:
def find_similarity(A,B):
  sim_mat = cosine_similarity(A, B)
  sim_add = 0
  for i in range(sim_mat.shape[0]):
      sim_add = sim_add + abs(sim_mat[i][i])
  return sim_add/sim_mat.shape[0]

In [12]:
from itertools import combinations
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from scipy import spatial
import matplotlib.pyplot as plt
import matplotlib
from sklearn.metrics.pairwise import cosine_similarity

def generate_samples(x_train,y_train,number_of_classes,Cparam,epsilon,samp_t,zi_type,eigvar):
  genDataTrainXFinal = np.empty(shape=[0, x_train.shape[1]])
  genDataTrainYFinal = np.empty(shape=[0,1])
  out_thres = 0.9 
  gen_mul_factor = 1
  gen_mul_factor_max = 10 #maximum number of times generated samples are computed
  in_thres = 0.98
  inner_counter = 0
  outer_counter = 0
  sim_outer_list = list()
  sim_inner_list = list() 
  while gen_mul_factor <= gen_mul_factor_max:
    print('outer_counter:'+str(outer_counter))
    print('inner_counter:'+str(inner_counter))
    if outer_counter%10==0 and outer_counter!=0:
      out_thres = np.mean(np.asarray(sim_outer_list))
      print('new out_thres:'+str(out_thres))
    if inner_counter%5==0 and inner_counter!=0:
      in_thres = np.mean(np.asarray(sim_inner_list))
      print('new in_thres:'+str(in_thres))
    for j in random_combination(range(x_train.shape[0]), samp_t, 1):
      data_train_used=x_train[j,:]
      labels_train_used=y_train[j,:]
      pca = PCA(n_components = 0.98)
      pca.fit(x_train)
      feature_weights_x_train = pca.components_
      n_components_for_x_train = feature_weights_x_train.shape[0]
      pca = PCA(n_components = n_components_for_x_train)
      pca.fit(data_train_used)
      feature_weights_data_train_used = pca.components_
      #finding similarity between the pca components of original dataset and dataset that will be used to generate samples
      sim = find_similarity(feature_weights_x_train,feature_weights_data_train_used)
      print('similarity:'+str(sim))
      sim_outer_list.append(abs(sim))
      outer_counter += 1
      if abs(sim)> out_thres:
        genDataX,genDataY = AugmentDataset(data_train_used,labels_train_used,Cparam,epsilon,zi_type,eigvar) 
        pca = PCA(n_components = 0.98)
        pca.fit(data_train_used)
        feature_weights_data_train_used = pca.components_
        n_components_for_data_train_used = feature_weights_data_train_used.shape[0]
        pca = PCA(n_components = n_components_for_data_train_used)
        pca.fit(genDataX)
        feature_weights_gen = pca.components_
        #finding similarity between the pca components of dataset being used to generate samples and the generated samples
        sim = find_similarity(feature_weights_data_train_used,feature_weights_gen)
        print('similarity.........:'+str(sim))
        sim_inner_list.append(abs(sim))
        inner_counter += 1
        if abs(sim)>in_thres:
          genDataTrainXFinal=np.append(genDataTrainXFinal,genDataX,axis=0)
          genDataTrainYFinal=np.append(genDataTrainYFinal,genDataY,axis=0)
          print('Added')
          print('generated_data_size:'+str(genDataTrainXFinal.shape[0]))
          gen_mul_factor+=1
        else:
          print('Discarded')
          print('generated_data_size:'+str(genDataTrainXFinal.shape[0]))
  return genDataTrainXFinal,genDataTrainYFinal

In [13]:
import tempfile
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
# from keras.optimizers import Adam
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
def train_test(orig_X,orig_Y,generated_X,generated_Y,\
               total_X,total_Y,test_X,test_Y):
  sc = StandardScaler().fit(orig_X)
  orig_X_sc = sc.transform(orig_X)
  test_X_1_sc = sc.transform(test_X)

  sc = StandardScaler().fit(total_X)
  total_X_sc = sc.transform(total_X)
  test_X_3_sc = sc.transform(test_X)

  no_of_classes = len(np.unique(y.reshape(-1)))
  encoded_orig_Y = orig_Y
  dummy_orig_Y = np_utils.to_categorical(encoded_orig_Y,num_classes=no_of_classes)
  encoded_total_Y = total_Y
  dummy_total_Y = np_utils.to_categorical(encoded_total_Y,num_classes=no_of_classes)
  encoded_test_Y = test_Y
  dummy_test_Y = np_utils.to_categorical(encoded_test_Y,num_classes=no_of_classes)

  model1 = Sequential()
  model1.add(Dense(1.5*orig_X.shape[1], input_dim=orig_X.shape[1], activation='relu',kernel_initializer=keras.initializers.RandomNormal(seed=1337),\
                   bias_initializer=keras.initializers.Constant(value=0.1)))
  model1.add(Dense(orig_X.shape[1], activation='relu',kernel_initializer=keras.initializers.RandomNormal(seed=1337),\
                   bias_initializer=keras.initializers.Constant(value=0.1)))
  model1.add(Dense(no_of_classes, activation='softmax',kernel_initializer=keras.initializers.RandomNormal(seed=1337),\
                   bias_initializer=keras.initializers.Constant(value=0.1)))
	# print(model1.summary())
  # Compile model
  model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  es1 = EarlyStopping(monitor='val_accuracy', mode='max', patience=10)
  mc1 = ModelCheckpoint('best_model1.h5', monitor='val_accuracy', mode='max', save_best_only=True)
  history=model1.fit(orig_X_sc,dummy_orig_Y,validation_data=(test_X_1_sc, dummy_test_Y),epochs=50,batch_size=min(int(0.2*orig_X.shape[0]),100),callbacks=[es1, mc1])
  saved_model1 = load_model('best_model1.h5')
  _, accuracy_orig = saved_model1.evaluate(test_X_1_sc, dummy_test_Y)
  print('Accuracy_orig: %.2f' % (accuracy_orig*100))
    
  model3 = Sequential()
  model3.add(Dense(1.5*orig_X.shape[1], input_dim=orig_X.shape[1], activation='relu',kernel_initializer=keras.initializers.RandomNormal(seed=1337),\
                   bias_initializer=keras.initializers.Constant(value=0.1)))
  model3.add(Dense(orig_X.shape[1], activation='relu',kernel_initializer=keras.initializers.RandomNormal(seed=1337),\
                   bias_initializer=keras.initializers.Constant(value=0.1)))
  model3.add(Dense(no_of_classes, activation='softmax',kernel_initializer=keras.initializers.RandomNormal(seed=1337),\
                   bias_initializer=keras.initializers.Constant(value=0.1)))
	# print(model3.summary())
  # Compile model
  model3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  es3 = EarlyStopping(monitor='val_accuracy', mode='max', patience=10)
  mc3 = ModelCheckpoint('best_model3.h5', monitor='val_accuracy', mode='max', save_best_only=True)
  history=model3.fit(total_X_sc,dummy_total_Y,validation_data=(test_X_3_sc, dummy_test_Y),epochs=50,batch_size=min(int(0.2*total_X.shape[0]),100),callbacks=[es3, mc3])
  saved_model3 = load_model('best_model3.h5')
  _, accuracy_tot = saved_model3.evaluate(test_X_3_sc, dummy_test_Y)
  print('Accuracy_tot: %.2f' % (accuracy_tot*100))

  return accuracy_orig,accuracy_tot

Parameters that can be changed:
1. Cparam
2. epsilon
3. threshold value(outer and inner)
4. gen_mul_factor
5. samples at a time
6. eigenvar


In [ ]:
from scipy.io import loadmat
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
import math
from sklearn.model_selection import train_test_split
#Required files:
#*.mat -> containing features(X) and classes(y)
#*fold.mat -> containing the indices of each fold
#*info.mat -> containg an array of the feature size denoting whether a feature is int valued(1) or real valued(0) 
# Define parameters for EigenSample - these can be changed according to the dataset
Cparam = 1
epsilon = 0.01
dataset_list=[1]#[13,16,19,2,21,23,24,26,27,5]
iter_lim = 1e2 #GRB.INFINITY

for dataset in dataset_list:
  Results = list()
  Results_arr = list()
  print('dataset:'+str(dataset))
  #annots = loadmat('{}.mat'.format(dataset))
  #annotsi = loadmat('{}fold.mat'.format(dataset))
  
  #annotsinf = loadmat('{}info.mat'.format(dataset))

  #X = annots['x']
  #y = annots['y']
  X = np.array(X.view(len(y),-1)) #TODO reshape(-1,1) if 1D
  y = np.array(y.view(-1,1)) #TODO reshape(-1,1) if 1D
  #print(X.shape)
  #print(y)
  #indices = annotsi['indices']
  #print(indices.shape)
  
  #indices = indices.reshape(indices.shape[0])
  #print(indices)

  indices = np.random.randint(1,3,(len(y))) #TODO just randonmly make it with batch index values (till maybe 3/4); same size as y
  #indices = indc
  #zi_type = annotsinf['zi_type']
  zi_type = np.zeros([X.shape[1]])
  y = y.reshape(y.shape[0])
  for i in range(y.shape[0]):
    if y[i] == -1:
      y[i] = 0  

  y_fold_min = math.inf
  for i in np.unique(indices):
    y_fold=y[indices!=i]
    y_fold_min = min(y_fold_min,y_fold.shape[0])

  for samples_at_a_time in [y_fold_min-1]:#TODO [y_fold_min-2,y_fold_min-5] keep it in this range (or even y_fold_min)
    for eigenvariance in [1]:#[1,0.98,0.95,0.9]:
      acc_orig_list = list()
      acc_tot_list = list()
      x_tr_list = list()
      tot_x_list = list() 
      y = y.reshape(-1,1)
      count = 1
      for i in np.unique(indices):
        X_Te = X[indices==i]
        y_Te = y[indices==i]
        X_Tr = X[indices!=i]
        y_Tr = y[indices!=i]

        gen_X,gen_y= generate_samples(X_Tr,y_Tr,len(np.unique(y_Tr.reshape(-1))),Cparam, epsilon, \
                                      samples_at_a_time,zi_type,eigenvariance)
        
        print("gen_X_shape:"+str(gen_X.shape))
        print("gen_y_shape:"+str(gen_y.shape))
        tot_X=np.append(X_Tr,gen_X,axis=0)
        tot_y=np.append(y_Tr,gen_y,axis=0)
        
        if count == 1:
            X_final = tot_X
            Y_final = tot_y
        else:
            X_final = np.append(X_final, tot_X, axis = 0)
            Y_final = np.append(Y_final, tot_y, axis = 0)
        count = count+1
        
        print("tot_X_shape:"+str(tot_X.shape))
        print("tot_y_shape:"+str(tot_y.shape))
        x_tr_list.append(X_Tr.shape[0])
        tot_x_list.append(tot_X.shape[0])
        try:
          acc_orig,acc_tot=train_test(X_Tr,y_Tr,gen_X,gen_y,tot_X,tot_y,X_Te,y_Te)
          acc_orig_list.append(acc_orig)
          acc_tot_list.append(acc_tot)
        except:
          print('except_occured')
      
      acc_orig_arr = np.array(acc_orig_list)
      acc_tot_arr = np.array(acc_tot_list)
      acc_orig_mean = np.mean(acc_orig_arr)
      acc_tot_mean = np.mean(acc_tot_arr)
      acc_orig_std = np.std(acc_orig_arr)
      acc_tot_std = np.std(acc_tot_arr)
      
      dic = dict()
      li = list()
      dic['dataset'] = dataset
      li.append(dataset)
      dic['parameter'] = samples_at_a_time
      li.append(samples_at_a_time)
      dic['eigenvar'] = eigenvariance
      li.append(eigenvariance)
      dic['orig_shape_0'] = np.mean(np.array(x_tr_list))
      li.append(X_Tr.shape[0])
      dic['orig_shape_1'] = X_Tr.shape[1]
      li.append(X_Tr.shape[1])
      dic['acc_orig_mean'] = acc_orig_mean
      li.append(acc_orig_mean)
      dic['acc_orig_std'] = acc_orig_std
      li.append(acc_orig_std)
      dic['orig_acc_fold'] = str(acc_orig_list).replace(',','/') 
      li.append(str(acc_orig_list).replace(',','/') )
      dic['tot_shape_0'] = np.mean(np.array(tot_x_list))
      li.append(tot_X.shape[0])
      dic['acc_tot_mean'] = acc_tot_mean
      li.append(acc_tot_mean)
      dic['acc_tot_std'] = acc_tot_std
      li.append(acc_tot_std)
      dic['tot_acc_fold'] = str(acc_tot_list).replace(',','/')  
      li.append(str(acc_tot_list).replace(',','/') )
      #np.savetxt("eigenbag_dataset_{}_samples_{}_eigenvar_{}.csv".format(dataset,samples_at_a_time,eigenvariance),np.array(li).reshape(1,len(li)),fmt= '%s',delimiter=',',\
      #       header='dataset, parameter, eigenvar, orig_shape_0, orig_shape_1, acc_orig_mean, acc_orig_std, orig_acc_fold, tot_shape_0, acc_to_mean, acc_tot_std, tot_acc_fold')
      Results.append(dic) 
      Results_arr.append(li)
  print("Results:")
  print(Results)
  Results_arr=np.array(Results_arr)
  np.savetxt("Results_eigenbag_{}.csv".format(dataset),Results_arr,fmt= '%s',delimiter=',',\
             header='dataset, parameter, eigenvar, orig_shape_0, orig_shape_1, acc_orig_mean, acc_orig_std, orig_acc_fold, tot_shape_0, acc_to_mean, acc_tot_std, tot_acc_fold')
             

dataset:1
outer_counter:0
inner_counter:0
similarity:0.9104901769791137
Set parameter Username
Academic license - for non-commercial use only - expires 2023-10-04
Set parameter IterationLimit to value 100
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 32 physical cores, 64 logical processors, using up to 32 threads
Optimize a model with 12288 rows, 9216 columns and 18860296 nonzeros
Model fingerprint: 0x27a84e13
Coefficient statistics:
  Matrix range     [1e-13, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e-05, 1e+01]

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Presolve removed 6144 rows and 0 columns (presolve time = 5s) ...
Presolve removed 6144 rows and 0 columns (presolve time = 14s) ...
Presolve removed 6144 rows and 0 columns (presolve time = 15s) ...
Presolve removed 6144 row

Presolve removed 6144 rows and 0 columns (presolve time = 20s) ...
Presolve removed 6144 rows and 0 columns
Presolve time: 22.70s
Presolved: 6144 rows, 9216 columns, 18324870 nonzeros


Stopped in 100 iterations and 30.31 seconds (16.80 work units)
Iteration limit reached
Set parameter IterationLimit to value 100
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 32 physical cores, 64 logical processors, using up to 32 threads
Optimize a model with 12288 rows, 9216 columns and 18860296 nonzeros
Model fingerprint: 0xa2231678
Coefficient statistics:
  Matrix range     [1e-13, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-05, 1e+01]

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Presolve removed 6144 rows and 0 columns (presolve time = 5s) ...
Presolve removed 6144 rows and 0 columns (presolv

Presolve removed 6144 rows and 0 columns (presolve time = 12s) ...
Presolve removed 6144 rows and 0 columns (presolve time = 15s) ...
Presolve removed 6144 rows and 0 columns
Presolve time: 17.34s
Presolved: 6144 rows, 9216 columns, 18324870 nonzeros

Ordering time: 0.19s

Barrier statistics:
 AA' NZ     : 6.667e+06
 Factor NZ  : 6.670e+06 (roughly 60 MB of memory)
 Factor Ops : 1.624e+10 (roughly 1 second per iteration)
 Threads    : 30

Barrier performed 0 iterations in 28.12 seconds (16.75 work units)
Barrier solve interrupted - model solved by another algorithm


Stopped in 100 iterations and 28.21 seconds (16.79 work units)
Iteration limit reached
Set parameter IterationLimit to value 100
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 32 physical cores, 64 logical processors, using up to 32 threads
Optimize a model with 12288 rows, 9216 columns and 18860296 nonzeros
Model fingerpr

Presolve removed 6144 rows and 0 columns (presolve time = 13s) ...
Presolve removed 6144 rows and 0 columns (presolve time = 15s) ...
Presolve removed 6144 rows and 0 columns
Presolve time: 17.73s
Presolved: 6144 rows, 9216 columns, 18324870 nonzeros


Barrier performed 0 iterations in 31.38 seconds (16.70 work units)
Barrier solve interrupted - model solved by another algorithm


Stopped in 100 iterations and 31.45 seconds (16.80 work units)
Iteration limit reached
Set parameter IterationLimit to value 100
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 32 physical cores, 64 logical processors, using up to 32 threads
Optimize a model with 12288 rows, 9216 columns and 18860296 nonzeros
Model fingerprint: 0x71583c6d
Coefficient statistics:
  Matrix range     [1e-13, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e-05, 1e+01]

Concurrent LP

Thread count: 32 physical cores, 64 logical processors, using up to 32 threads
Optimize a model with 12288 rows, 9216 columns and 18860296 nonzeros
Model fingerprint: 0xf33ddb41
Coefficient statistics:
  Matrix range     [1e-13, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e-05, 1e+01]

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Presolve removed 6144 rows and 0 columns (presolve time = 6s) ...
Presolve removed 6144 rows and 0 columns (presolve time = 12s) ...
Presolve removed 6144 rows and 0 columns (presolve time = 16s) ...
Presolve removed 6144 rows and 0 columns
Presolve time: 18.30s
Presolved: 6144 rows, 9216 columns, 18324870 nonzeros


Barrier performed 0 iterations in 28.59 seconds (16.70 work units)
Barrier solve interrupted - model solved by another algorithm


Stopped in 100 iterations and 30.13 seconds (16.80 work units)
Iteration limit reached
Set parameter IterationLimit

...


Presolve removed 6144 rows and 0 columns (presolve time = 5s) ...
Presolve removed 6144 rows and 0 columns (presolve time = 12s) ...
Presolve removed 6144 rows and 0 columns (presolve time = 15s) ...
Presolve removed 6144 rows and 0 columns
Presolve time: 16.21s
Presolved: 6144 rows, 9216 columns, 18324870 nonzeros

Ordering time: 0.15s

Barrier statistics:
 AA' NZ     : 6.667e+06
 Factor NZ  : 6.670e+06 (roughly 60 MB of memory)
 Factor Ops : 1.624e+10 (less than 1 second per iteration)
 Threads    : 30

Barrier performed 0 iterations in 29.41 seconds (16.75 work units)
Barrier solve interrupted - model solved by another algorithm


Stopped in 100 iterations and 29.46 seconds (16.80 work units)
Iteration limit reached
Set parameter IterationLimit to value 100
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 32 physical cores, 64 logical processors, using up to 32 threads
Optimize a mod

 Threads    : 30

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.34349071e+04 -2.57711348e+02  8.35e+02 0.00e+00  2.08e+00    36s

Barrier performed 0 iterations in 39.20 seconds (22.42 work units)
Barrier solve interrupted - model solved by another algorithm


Stopped in 100 iterations and 39.35 seconds (16.80 work units)
Iteration limit reached
Set parameter IterationLimit to value 100
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 32 physical cores, 64 logical processors, using up to 32 threads
Optimize a model with 12288 rows, 9216 columns and 18860296 nonzeros
Model fingerprint: 0x5d645cc8
Coefficient statistics:
  Matrix range     [1e-13, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e-05, 8e+00]

Concurrent LP optimizer: primal simplex, dual s

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 32 physical cores, 64 logical processors, using up to 32 threads
Optimize a model with 12288 rows, 9216 columns and 18860296 nonzeros
Model fingerprint: 0x4e1f0f9b
Coefficient statistics:
  Matrix range     [1e-13, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-05, 9e+00]

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Presolve removed 6144 rows and 0 columns (presolve time = 5s) ...
Presolve removed 6144 rows and 0 columns (presolve time = 11s) ...
Presolve removed 6144 rows and 0 columns (presolve time = 15s) ...
Presolve removed 6144 rows and 0 columns
Presolve time: 16.84s
Presolved: 6144 rows, 9216 columns, 18324870 nonzeros

Ordering time: 0.16s

Barrier statistics:
 AA' NZ     : 6.667e+06
 Factor NZ  : 6.670e+06 (roughly 60 MB of memory)
 Factor Ops : 1.624e+10 (roughly 1 second per iteration)
 Threads    : 30


Model fingerprint: 0xcbb2598b
Coefficient statistics:
  Matrix range     [1e-13, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-05, 9e+00]

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Presolve removed 6144 rows and 0 columns (presolve time = 6s) ...
Presolve removed 6144 rows and 0 columns (presolve time = 14s) ...
Presolve removed 6144 rows and 0 columns (presolve time = 15s) ...
Presolve removed 6144 rows and 0 columns
Presolve time: 18.78s
Presolved: 6144 rows, 9216 columns, 18324870 nonzeros


Stopped in 100 iterations and 27.64 seconds (16.80 work units)
Iteration limit reached
Set parameter IterationLimit to value 100
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 32 physical cores, 64 logical processors, using up to 32 threads
Optimize a model with 12288 rows, 9216 columns and 

Iteration limit reached
Set parameter IterationLimit to value 100
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 32 physical cores, 64 logical processors, using up to 32 threads
Optimize a model with 12288 rows, 9216 columns and 18860296 nonzeros
Model fingerprint: 0x3078a823
Coefficient statistics:
  Matrix range     [1e-13, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e-06, 8e+00]

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Presolve removed 6144 rows and 0 columns (presolve time = 5s) ...
Presolve removed 6144 rows and 0 columns (presolve time = 14s) ...
Presolve removed 6144 rows and 0 columns (presolve time = 15s) ...
Presolve removed 6144 rows and 0 columns
Presolve time: 19.68s
Presolved: 6144 rows, 9216 columns, 18324870 nonzeros

Ordering time: 0.23s

Barrier statistics:
 AA'

In [ ]:
(X_final).shape

In [ ]:
(Y_final).shape

In [ ]:
X_final = X_final.reshape(-1, 3, imgsize, imgsize)

In [ ]:
np.save(f'EigenData/DrVineet_X_{imgsize}', X_final)

In [ ]:
np.save(f'EigenData/DrVineet_y_{imgsize}', Y_final)